In [ ]:
# 本脚本用于调用deepseek v3的api，从而为FlakyLen的数据集增加推理过程


In [ ]:
import pandas as pd
import json
import os
from openai import OpenAI
from tqdm import tqdm
import time

# 读取CSV文件
csv_file = 'FlakyLens_dataset_with_nonflaky_indented.csv'
df = pd.read_csv(csv_file)

# 查看数据格式
print(f"数据集大小: {len(df)} 条")
print(f"\n列名: {df.columns.tolist()}")
print(f"\n前几行数据:")
print(df.head())
print(f"\n数据类型:")
print(df.dtypes)

In [ ]:
# 初始化DeepSeek API客户端
# 请替换为你的API密钥
API_KEY = "your-deepseek-api-key-here"
BASE_URL = "https://api.deepseek.com"

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

In [ ]:
def generate_reasoning_prompt(row):
    """
    根据数据行生成prompt，让模型分析并生成推理过程
    需要根据实际的CSV列名调整
    """
    # 这里假设CSV有code、test_name、label等列
    # 请根据实际的列名调整
    
    prompt = f"""请分析以下测试用例，判断它是否是一个Flaky Test（不稳定测试），并详细说明你的推理过程。

测试代码：
{row.get('code', row.get('test_code', ''))}

请按照以下格式回答：
1. 分析测试代码的关键特征
2. 识别可能导致测试不稳定的因素（如时间依赖、并发问题、外部依赖等）
3. 给出最终判断：是否为Flaky Test（回答"是"或"否"）
4. 解释你是如何得出这个结论的

请确保推理过程清晰、有逻辑。"""
    
    return prompt

def call_deepseek_api(prompt, max_retries=3):
    """
    调用DeepSeek API生成推理过程
    """
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "你是一个专业的软件测试专家，擅长分析测试代码并识别Flaky Tests。"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=2000
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"API调用失败 (尝试 {attempt + 1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # 指数退避
            else:
                return None
    return None

In [ ]:
def convert_to_alpaca_format(row, reasoning):
    """
    将数据转换为Alpaca格式
    Alpaca格式: {instruction, input, output}
    """
    # 根据实际CSV列名调整
    test_code = row.get('code', row.get('test_code', ''))
    label = row.get('label', row.get('is_flaky', ''))
    
    # 构建instruction
    instruction = "请分析以下测试用例，判断它是否是一个Flaky Test（不稳定测试），并说明你的推理过程。"
    
    # input是测试代码
    input_text = f"测试代码：\n{test_code}"
    
    # output是模型生成的推理过程
    output_text = reasoning
    
    alpaca_item = {
        "instruction": instruction,
        "input": input_text,
        "output": output_text
    }
    
    return alpaca_item

In [ ]:
# 主处理流程
distilled_dataset = []
failed_indices = []

print(f"开始处理 {len(df)} 条数据...")
print("=" * 50)

for idx, row in tqdm(df.iterrows(), total=len(df)):
    # 生成prompt
    prompt = generate_reasoning_prompt(row)
    
    # 调用API获取推理过程
    reasoning = call_deepseek_api(prompt)
    
    if reasoning is None:
        print(f"\n警告: 第 {idx} 条数据处理失败")
        failed_indices.append(idx)
        continue
    
    # 转换为Alpaca格式
    alpaca_item = convert_to_alpaca_format(row, reasoning)
    distilled_dataset.append(alpaca_item)
    
    # 每处理10条数据暂停一下，避免API限流
    if (idx + 1) % 10 == 0:
        time.sleep(1)
    
    # 每处理50条数据保存一次中间结果
    if (idx + 1) % 50 == 0:
        with open('distillation_dataset_temp.json', 'w', encoding='utf-8') as f:
            json.dump(distilled_dataset, f, ensure_ascii=False, indent=2)
        print(f"\n已处理 {idx + 1} 条，中间结果已保存")

print("\n" + "=" * 50)
print(f"处理完成！成功: {len(distilled_dataset)} 条，失败: {len(failed_indices)} 条")

In [ ]:
# 保存最终的蒸馏数据集
output_file = 'distillation_dataset.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(distilled_dataset, f, ensure_ascii=False, indent=2)

print(f"\n蒸馏数据集已保存到: {output_file}")
print(f"总共 {len(distilled_dataset)} 条数据")

# 显示一个示例
if distilled_dataset:
    print("\n示例数据:")
    print("=" * 50)
    print(json.dumps(distilled_dataset[0], ensure_ascii=False, indent=2))

## 使用说明

1. **准备工作**：
   - 确保 `FlakyLens_dataset_with_nonflaky_indented.csv` 文件在同一目录下
   - 在第3个单元格中填入你的 DeepSeek API Key
   - 安装必要的依赖：`pip install pandas openai tqdm`

2. **运行流程**：
   - 先运行第2个单元格，查看CSV文件的格式和列名
   - 根据实际的列名，调整第4个单元格中的 `generate_reasoning_prompt` 函数
   - 同样调整第5个单元格中的 `convert_to_alpaca_format` 函数
   - 运行第6个单元格开始处理数据
   - 运行第7个单元格保存最终结果

3. **注意事项**：
   - 脚本会每50条数据保存一次中间结果到 `distillation_dataset_temp.json`
   - 如果中断，可以从中间结果继续处理
   - API调用有重试机制，失败的数据会被记录
   - 每10条数据会暂停1秒，避免触发API限流